In [1]:
import os
import pandas as pd
from feature_functions import apply_zscore_signal
from utils.backtester import Backtester

# Directory containing asset data
asset_dir = './hist_data/crypto/kraken_4hr/'

# Initialize EV dictionary
asset_ev_dict = {}
df_dict = {}
reg_window = 5
reg_thresh = .75
# Iterate over all assets
for asset_file in os.listdir(asset_dir):
    if asset_file.endswith('.txt'):
        asset_path = os.path.join(asset_dir, asset_file)
        df = pd.read_csv(asset_path, sep=',')
        df['Date'] = pd.to_datetime(df['date'], unit='s')
        df_dict[asset_file] = df

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

keys = list(df_dict.keys())

test_df = df_dict[keys[0]]


returns = test_df['close'].pct_change().dropna()

# Plot ACF and PACF
plt.figure(figsize=(12, 6))
plot_acf(returns, lags=20, title="ACF of Returns")
plot_pacf(returns, lags=20, title="PACF of Returns")
plt.show()